In [1]:
'''

啟動伺服器基本樣版

設定業務功能
    當用戶關注時
        記錄個資，綁定菜單，回傳文字消息與圖片消息，介紹功能與使用方法
    當用戶點擊自定義菜單或輸入文字消息時
        傳送圖片消息或相應文字消息。
    當用戶點擊按鈕消息的時候
        會將用戶個資紀錄回檔案內，以供未來聯繫用

'''

'\n\n啟動伺服器基本樣版\n\n設定業務功能\n    當用戶關注時\n        記錄個資，綁定菜單，回傳文字消息與圖片消息，介紹功能與使用方法\n    當用戶點擊自定義菜單或輸入文字消息時\n        傳送圖片消息或相應文字消息。\n    當用戶點擊按鈕消息的時候\n        會將用戶個資紀錄回檔案內，以供未來聯繫用\n\n'

In [2]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
# '''

# 製作文字與圖片的教學訊息

# '''
# # 將消息模型，文字收取消息與文字寄發消息 引入
# from linebot.models import (
#     MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
# )

# # 消息清單
# reply_message_list = [
# TextSendMessage(text="了解國際，放眼未來，從觀看國際新聞開始"),
# TextSendMessage(text="使用此LineBot，能得永生，你信嗎")
# ]



In [4]:
"""


當用戶關注Line@後，Line會發一個FollowEvent，

我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳消息給用戶


"""


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    
    
    # 消息清單
    reply_message_list = [
    TextSendMessage(text="了解國際，放眼未來，從觀看國際新聞開始"),
    TextSendMessage(text="使用此LineBot，能得永生，你信嗎")
    ]

    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [ ]:
'''

製作模板

'''
flexCarouselContainerJsonDict = """
{
"type": "carousel",
    "contents": [
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/01_5_carousel.png",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "%s",
              "size": "xl",
              "weight": "bold",
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "contents": [
                {
                  "type": "text",
                  "text": "view:",
                  "flex": 0,
                  "size": "sm",
                  "gravity": "top",
                  "weight": "bold",
                  "wrap": true
                },
                {
                  "type": "text",
                  "text": "???",
                  "flex": 0,
                  "size": "sm",
                  "weight": "bold",
                  "wrap": true
                }
              ]
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "內文點我",
                "uri": "https://linecorp.com"
              },
              "color": "#52AC2F",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "喜歡",
                "data": "%s"
              }
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "不喜歡",
                "data": "%s"
              }
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/01_6_carousel.png",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "%s",
              "size": "xl",
              "weight": "bold",
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "view:",
                  "flex": 0,
                  "size": "sm",
                  "weight": "bold",
                  "wrap": true
                },
                {
                  "type": "text",
                  "text": "???",
                  "flex": 0,
                  "size": "sm",
                  "weight": "bold",
                  "wrap": true
                }
              ]
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "內文點我",
                "uri": "https://linecorp.com"
              },
              "flex": 2,
              "color": "#52AC2F",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "喜歡",
                "data": "%s"
              }
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "不喜歡",
                "data": "%s"
              }
            }
          ]
        }
      },
      {
        "type": "bubble",
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "See more",
                "uri": "https://linecorp.com"
              },
              "flex": 1,
              "gravity": "center"
            }
          ]
        }
      }
    ]
  }
"""%("newstitle1","newstitle1[like]","newstitle[dislike]1","newstitle2","newstitle2[like]","newstitle2[dislike]")

In [ ]:
from linebot.models import(
    FlexSendMessage,CarouselContainer,BubbleContainer
)

import json

carouselContent = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict))
flexCarouselSendMeesage =  FlexSendMessage(alt_text="hello", contents=carouselContent)

In [ ]:
'''

建立圖片消息素材

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    TextSendMessage
)

# 圖片消息的基本建構特徵
text_message = TextSendMessage(text = "??")
text_message2 = TextSendMessage(text = "功能尚在開發")
text_message3 = TextSendMessage(text = "開發者:\n吳冠廷\n陳俊龍\n王楷欣\n陳重叡\n胡元亨\n郝學偉")

In [ ]:
'''
將素材消息做成一本字典，當用戶發出相應消息時，可從此進行查找動作。
'''
template_message_dict = {
    "!關鍵字搜尋":text_message,
    "!人氣熱度預測":text_message2,
    "!訂閱":text_message2,
    "!新聞推薦":flexCarouselSendMeesage,
    "!國外熱門新聞":flexCarouselSendMeesage,
    "!關於我們":text_message3
}

In [ ]:
'''

當用戶發出文字消息時，判斷文字內容是否包含!，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''
    

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print(event)
    if(event.message.text.find('!')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text))
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊選單了解更多"))

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from linebot.models import (
     PostbackEvent
)
# @handler.add(PastbackEvent)
# def handle_post_message(event):
#     user_profile = Line_bot_api_profile(event.source.user_id)
#     print(event)
#     with open("../user_profile_business.txt","a") as myfile:
        
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    print(event)
    print("---------------------")
    print('{"data":"%s","userid":"%s"}'%(event.postback.data,event.source.user_id))
    if (event.postback.data.find('0')== 0):
        with open("../user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\r\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='喜歡的話之後會增加此類型出現'
                )
            )
    elif (event.postback.data.find('1') == 0):
        with open("../user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\r\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='不喜歡的話之後會減少此類型出現'
                )
            )
    else:
        pass

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


{"message": {"id": "9450051121452", "text": "!\u570b\u5916\u71b1\u9580\u65b0\u805e", "type": "text"}, "replyToken": "5eaab93fb37f4f989501862d1e31e4ac", "source": {"type": "user", "userId": "Ub2229d6c802d8c3d334cdcc8b4e494a9"}, "timestamp": 1551610701545, "type": "message"}


172.17.0.3 - - [03/Mar/2019 10:58:23] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Mar/2019 10:58:25] "POST / HTTP/1.1" 200 -


{"postback": {"data": "newstitle1[like]"}, "replyToken": "8f208baca12341bdb25128278ef6a614", "source": {"type": "user", "userId": "Ub2229d6c802d8c3d334cdcc8b4e494a9"}, "timestamp": 1551610704856, "type": "postback"}
---------------------
{"data":"newstitle1[like]","userid":"Ub2229d6c802d8c3d334cdcc8b4e494a9"}


172.17.0.3 - - [03/Mar/2019 10:58:29] "POST / HTTP/1.1" 200 -


{"postback": {"data": "newstitle[dislike]1"}, "replyToken": "63bcd5b336e747ec972e1200be4bb497", "source": {"type": "user", "userId": "Ub2229d6c802d8c3d334cdcc8b4e494a9"}, "timestamp": 1551610709016, "type": "postback"}
---------------------
{"data":"newstitle[dislike]1","userid":"Ub2229d6c802d8c3d334cdcc8b4e494a9"}


172.17.0.3 - - [03/Mar/2019 10:58:35] "POST / HTTP/1.1" 200 -


{"postback": {"data": "newstitle[dislike]1"}, "replyToken": "b8caaba4f96b4808ba0662d732ce0eac", "source": {"type": "user", "userId": "Ub2229d6c802d8c3d334cdcc8b4e494a9"}, "timestamp": 1551610714550, "type": "postback"}
---------------------
{"data":"newstitle[dislike]1","userid":"Ub2229d6c802d8c3d334cdcc8b4e494a9"}


172.17.0.3 - - [03/Mar/2019 10:58:39] "POST / HTTP/1.1" 200 -


{"postback": {"data": "newstitle2[dislike]"}, "replyToken": "a85b009d090f479a8a94154c8f4d7dce", "source": {"type": "user", "userId": "Ub2229d6c802d8c3d334cdcc8b4e494a9"}, "timestamp": 1551610718216, "type": "postback"}
---------------------
{"data":"newstitle2[dislike]","userid":"Ub2229d6c802d8c3d334cdcc8b4e494a9"}
